In [18]:
import torch
import torchvision
import torchvision.transforms as transforms

device = torch.cuda.is_available()


In [30]:
# Hyperparameters

episodes = 10
batch_size = 16
n_labels = 10
learning_rate = 1e-3
debug_rate = 1

In [31]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,) * 3, (0.5,) * 3)
])

train_set = torchvision.datasets.CIFAR10(
    root='./cifardata', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, num_workers=2)

Files already downloaded and verified


Files already downloaded and verified


In [32]:
from torch.autograd import Variable
import torch.nn.functional as F


class SimpleCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, X):
        a1 = F.relu(self.conv1(X))
        a2 = self.pool(a1)
        flattened = a2.view(-1, 18 * 16 * 16)
        a3 = F.relu(self.fc1(flattened))
        h = F.relu(self.fc2(a3))
        return h


In [33]:
net = SimpleCNN()
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for episode in range(episodes):
    curr_loss = 0
    for batch, labels in train_loader:
        X = Variable(batch)
        y = Variable(labels)
        optimizer.zero_grad()
        
        outputs = net(X)
        batch_loss = loss(outputs, labels)
        batch_loss.backward()
        optimizer.step()
        
        curr_loss += batch_loss
    
    print(f'Episode {episode}: Loss={curr_loss}')

Episode 0: Loss=5394.60302734375


Episode 1: Loss=4604.2158203125


Episode 2: Loss=4310.85888671875


Episode 3: Loss=4096.8583984375


Episode 4: Loss=3930.874755859375


KeyboardInterrupt: 